In [1]:
from __future__ import print_function
import cv2
import os
import glob
import numpy as np


import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import img_to_array
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam
from keras.regularizers import l2
from sklearn.model_selection import StratifiedShuffleSplit

Using TensorFlow backend.
/home/alexander/anaconda3/envs/tf/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/alexander/anaconda3/envs/tf/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/alexander/anaconda3/envs/tf/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/alexander/anacond

In [2]:
def load_data(categories):
    # Training and Testing
    data             = []
    target           = []
    size_image       = 32
    img_dir          = 'train'    
    
    for idx, category in enumerate(categories):
        img_dir_aux = img_dir + '/'+ category
        data_path   = os.path.join(img_dir_aux, '*g')
        files       = glob.glob(data_path)
        for f1 in files:
            img        = cv2.imread(f1, cv2.IMREAD_GRAYSCALE)
            img_resize = cv2.resize(img, (size_image, size_image), interpolation = cv2.INTER_AREA)
            img_resize = img_to_array(img_resize)
            
            data.append(img_resize)
            target.append(idx)
        
    target     = keras.utils.to_categorical(target, len(categories))    
    data       = np.array(data)
    target     = np.array(target)    
    data       = data.astype(np.uint8)/255.0 
    
    aux          = np.arange(len(data))
    np.random.shuffle(aux)
    train_data   = data[aux]
    train_target = target[aux]
        
    # Validation
    val_data   = []
    val_target = []
    img_dir    = 'val'
    for idx, category in enumerate(categories):        
        img_dir_aux = img_dir + '/'+ category
        data_path   = os.path.join(img_dir_aux, '*g')
        files       = glob.glob(data_path)        
        for f1 in files:
            img        = cv2.imread(f1, cv2.IMREAD_GRAYSCALE)
            img_resize = cv2.resize(img, (size_image, size_image), interpolation = cv2.INTER_AREA)
            img_resize = img_to_array(img_resize)
            val_data.append(img_resize)
            val_target.append(idx)
    
    val_target   = keras.utils.to_categorical(val_target, len(categories))                
    val_data     = np.array(val_data)
    
    val_data =val_data.astype(np.uint8)/255.0
    
    val_target   = np.array(val_target)        
    return train_data, train_target, val_data, val_target

In [3]:
batch_size        = 32
num_classes       = 7
save_dir          = os.path.join(os.getcwd(), 'saved_models')
model_name        = 'faces.h5'
categories  = ['angry','disgust','fear','happy','neutral','sad','surprise']
# categories  = ['angry','disgust']
# train_data, train_target, test_data, test_target, val_data, val_target = load_data(categories)
train_data, train_target, val_data, val_target = load_data(categories)

print('train_data.shape[1:]')
print(train_data.shape[1:])

print(len(train_data))
print(len(train_target))

train_data.shape[1:]
(32, 32, 1)
2910
2910


In [4]:
epochs            = 50

In [5]:
model = Sequential()
model.add(Conv2D(32, (5, 5), activation = 'relu', padding='same',input_shape=train_data.shape[1:]))
model.add(Conv2D(32, (5, 5), activation = 'relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))

model.add(Conv2D(64, (5, 5), padding='same', activation = 'relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))

model.add(Flatten())
model.add(Dense(512, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(256, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

# initiate RMSprop optimizer
# ACHF - se cambia el nombre del parametro, nueva documentacion
# opt = keras.optimizers.RMSprop(lr = 0.0001, decay = 1e-6)
opt = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-7)
# opt = keras.optimizers.RMSprop(lr = 0.001)

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',optimizer=opt,metrics=['accuracy'])
# model.summary()

batch_size = 32
print(val_data.shape)
model.fit(train_data,train_target,epochs = epochs, batch_size = batch_size,validation_data = (val_data, val_target), shuffle = True)









Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.

(1000, 32, 32, 1)
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 2910 samples, validate on 1000 samples
Epoch 1/50
2910/2910 [==============================] - 11s 4ms/step - loss: 3.0812 - acc: 0.1993 - val_loss: 1.8032 - val_acc: 0.2680
Epoch 2/50
2910/2910 [==============================] - 7s 2ms/step - loss: 2.0398 - acc: 0.2223 - val_loss: 1.7766 - val_acc: 0.4670
Epoch 3/50
2910/2910 [==============================] - 6s 2ms/step - loss: 1.8875 - acc: 0.2663 - val_loss: 1.8017 - val_acc: 0.3590
Epoch 4/50
2910/2910 [==============================] - 11s 4ms/step - loss: 1.8008 - acc: 0.3086 - val_loss: 1.5629 - val_acc: 0.4750
Epoch 5/50
2910/2910 [==============================] - 9s 3ms/step - loss: 1.7439 - acc: 0.3282 - val_loss: 1.5915 - val_acc: 0.4700
Epoch 6/50
2910/2910 [=======================

2910/2910 [==============================] - 7s 2ms/step - loss: 0.7236 - acc: 0.7430 - val_loss: 1.2862 - val_acc: 0.5930
Epoch 44/50
2910/2910 [==============================] - 8s 3ms/step - loss: 0.6533 - acc: 0.7722 - val_loss: 1.1389 - val_acc: 0.6420
Epoch 45/50
2910/2910 [==============================] - 7s 2ms/step - loss: 0.6577 - acc: 0.7625 - val_loss: 1.2170 - val_acc: 0.6270
Epoch 46/50
2910/2910 [==============================] - 7s 2ms/step - loss: 0.6254 - acc: 0.7852 - val_loss: 1.1598 - val_acc: 0.6070
Epoch 47/50
2910/2910 [==============================] - 7s 3ms/step - loss: 0.6088 - acc: 0.7797 - val_loss: 1.2555 - val_acc: 0.6150
Epoch 48/50
2910/2910 [==============================] - 7s 3ms/step - loss: 0.6160 - acc: 0.7832 - val_loss: 1.2492 - val_acc: 0.6110
Epoch 49/50
2910/2910 [==============================] - 8s 3ms/step - loss: 0.5909 - acc: 0.7969 - val_loss: 1.2339 - val_acc: 0.6080
Epoch 50/50
2910/2910 [==============================] - 8s 3ms/ste

In [6]:
model_name        = 'facesv6.h5'
# Save model and weights
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('Saved trained model at %s ' % model_path)

# train_data, train_target, test_data, test_target, val_data, val_target = load_data(categories)

# Score trained model.
scores = model.evaluate(test_data, test_target, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

scores = model.evaluate(val_data, val_target, verbose=1)
print('Val loss:', scores[0])
print('Val accuracy:', scores[1])

Saved trained model at /home/alexander/Documents/Cursos/RN/Tareas/Projeto2/saved_models/facesv6.h5 


NameError: name 'test_data' is not defined

In [ ]:
# model = Sequential()
# model.add(Conv2D(64, (3, 3), activation='relu', input_shape=train_data.shape[1:], kernel_regularizer=l2(0.01)))
# model.add(Conv2D(64, (3, 3), padding='same',activation='relu'))
# model.add(BatchNormalization())
# model.add(MaxPooling2D(pool_size=(2,2), strides=(2, 2)))
# model.add(Dropout(0.5))    
# model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
# model.add(BatchNormalization())
# model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
# model.add(BatchNormalization())
# model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
# model.add(BatchNormalization())
# model.add(MaxPooling2D(pool_size=(2,2)))
# model.add(Dropout(0.5))    
# model.add(Conv2D(256, (3, 3), padding='same', activation='relu'))
# model.add(BatchNormalization())
# model.add(Conv2D(256, (3, 3), padding='same', activation='relu'))
# model.add(BatchNormalization())
# model.add(Conv2D(256, (3, 3), padding='same', activation='relu'))
# model.add(BatchNormalization())
# model.add(MaxPooling2D(pool_size=(2,2)))
# model.add(Dropout(0.5))    
# model.add(Conv2D(512, (3, 3), padding='same', activation='relu'))
# model.add(BatchNormalization())
# model.add(Conv2D(512, (3, 3), padding='same', activation='relu'))
# model.add(BatchNormalization())
# model.add(Conv2D(512, (3, 3), padding='same', activation='relu'))
# model.add(BatchNormalization())
# model.add(MaxPooling2D(pool_size=(2,2)))
# model.add(Dropout(0.5))    
# model.add(Flatten())
# model.add(Dense(512, activation='relu'))
# model.add(Dropout(0.5))
# model.add(Dense(256, activation='relu'))
# model.add(Dropout(0.5))
# model.add(Dense(128, activation='relu'))
# model.add(Dropout(0.5))
# model.add(Dense(64, activation='relu'))
# model.add(Dropout(0.5))
# model.add(Dense(7, activation='softmax'))

# learning_rate = 0.001
# adam = Adam(lr = learning_rate)
# model.compile(optimizer = adam, loss = 'categorical_crossentropy', metrics = ['accuracy'])

# batch_size = 32
# print(val_data.shape)
# model.fit(train_data,train_target,epochs = epochs, batch_size = batch_size,validation_data = (val_data, val_target), shuffle = True)